In [1]:
!pip install pandasai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 13.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.6 MB/s eta 0:00:00m eta 0:00:010:01:01
  Created wheel for pandas: filename=pandas-1.5.3-cp312-cp312-linux_x86_64.whl size=11477855 sha256=153c0d06333cf1978b3e803964a99fab454ad77304db7c4767ddb1c6fa333073
  Stored in directory: /home/bachngo/.cache/pip/wheels/fb/83/18/8e7307aa1185c5498c5490e4d9c8a1732d9f1056e86c3491c6
Successfully built pandas
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolve

In [12]:
!pip install seaborn

zsh:1: no matches found: pandasai[extra-dependency-name]


In [1]:
from pandasai import SmartDataframe
from pandasai.llm.local_llm import LocalLLM
from pandasai.llm import OpenAI

In [2]:
# llm = LocalLLM(
#     api_base="https://api.groq.com/openai/v1",
#     model="llama3-8b-8192"
# )

llm = OpenAI(
    api_token="",
)

In [3]:
import pandas as pd
from pandasai import Agent

df = pd.read_csv("./data/train.csv")
agent = Agent(df, config={"llm": llm})

In [7]:
print(agent.chat("What is the feature with highest correlation in this data"))

The feature with the highest correlation with 'Microwave' is 'Gymnasium'.


<string>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
